# Projet 4 - Interpretation de BERT

Elyes KHALFALLAH & Edouard CHAPPON

MALIA

---

---


In [1]:
# # install required python packages
# !pip install seaborn
# !pip install pandas
# !pip install matplotlib
# !pip install transformers
# !pip install captum
# #mecab
# !apt install aptitude swig
# !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# !pip install mecab-python3==0.996.3

# #neologd
# # !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
# # !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

# !pip install transformers
# !pip install fugashi==1.1.0 ipadic==1.0.0


  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------- ----------------- 5.5/9.7 MB 30.5 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 28.7 MB/s eta 0:00:00
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Using cached captum-0.7.0-py3-none-any.whl.metadata (26 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.m

'apt' is not recognized as an internal or external command,
operable program or batch file.
'aptitude' is not recognized as an internal or external command,
operable program or batch file.


  Using cached mecab-python3-0.996.3.tar.gz (63 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [80 lines of output]
      C:\Users\elyes\Desktop\MALIA\XAI_Project\env_xai\Lib\site-packages\setuptools\__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      [02/08/25 23:06:30] WARNING  toml section missing       pyproject_reading.py:42
                                   'pyproject.toml does not
                                   contain a
                                   tool.setuptools_scm
              

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
     ----------- ---------------------------- 3.9/13.4 MB 19.6 MB/s eta 0:00:01
     ------------------------ --------------- 8.1/13.4 MB 19.4 MB/s eta 0:00:01
     ---------------------------------- ---- 11.8/13.4 MB 18.5 MB/s eta 0:00:01
     --------------------------------------- 13.4/13.4 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for fugashi
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556766 sha256=c5a82e967a1359e2f7fa25d44ec90d1823a5ff06610ebcbd1c5600a6e242b16d
  Stored in directory: c:\users\elyes\appdata\local\pip\cache\wheels\93\8b\55\dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
Successfully built ipadic
Failed to build fugashi


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      C:\Users\elyes\Desktop\MALIA\XAI_Project\env_xai\Lib\site-packages\setuptools\__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      [02/08/25 23:06:41] WARNING  toml section missing       pyproject_reading.py:42
                                   'pyproject.toml does not
                                   contain a
                                   tool.setuptools_scm
           

In [2]:
# import required python libraries
import numpy as np  # multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import pandas as pd  # data manipulation and analysis
import seaborn as sns  # statistical data visualization
import matplotlib.pyplot as plt  # object-oriented API for embedding plots

import torch  # data structures for multi-dimensional tensors and defines mathematical operations over these tensors
import torch.nn as nn  # basic building blocks for graphs
import json

# from transformers import pipeline

# library for transformer models
from transformers import (
    BertTokenizer,
    BertForQuestionAnswering,
    BertConfig,
)

# captum visualization tool for matplotlib figure, it visualizes attribution for a given image by normalizing attribution values of the desired sign
from captum.attr import (
    visualization as viz,
)

# computes conductance with respect to the given layer and provides integrated gradients augmenting accuracy metrics, model debugging and feature or rule extraction.
from captum.attr import (
    LayerConductance,
    LayerIntegratedGradients,
)


c:\Users\elyes\Desktop\MALIA\XAI_Project\env_xai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device


device(type='cpu')